#  Build an Interactive Dashboard with Plotly Dash

After visual analysis using the dashboard, you should be able to obtain some insights to answer the following five questions:

- Which site has the largest successful launches?
- Which site has the highest launch success rate?
- Which payload range(s) has the highest launch success rate?
- Which payload range(s) has the lowest launch success rate?
- Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest
launch success rate?

In [164]:
# Import Necessary Libraries
from dash import Dash, html, dcc, Input, Output
import jupyter_dash
from jupyter_dash.comms import _send_jupyter_config_comm_request 
import plotly.express as px
import pandas as pd
import wget

import warnings
warnings.filterwarnings("ignore")


In [165]:
# URL
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
wget.download(url, "spacex_launch_dash.csv")


'spacex_launch_dash (14).csv'

In [166]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df["Payload Mass (kg)"].max()
min_payload = spacex_df["Payload Mass (kg)"].min()

### Create a dash application


In [167]:
app = dash.Dash(__name__)

### Create an app layout


#### TASK 1: Add a Launch Site Drop-down Input Component

In [168]:
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center',
                'color': '#503D36',
                'font-size': 40}),

    # TASK 1: Add a dropdown list to enable Launch Site selection
    dcc.Dropdown(
        id="site-dropdown",
        options=[
            {"label": "All Sites", "value": "ALL"},
            {"label": "CCAFS LC-40", "value": "CCAFS LC-40"},
            {"label": "VAFB SLC-4E", "value": "VAFB SLC-4E"},
            {"label": "KSC LC-39A", "value": "KSC LC-39A"},
            {"label": "CCAFS SLC-40", "value": "CCAFS SLC-40"}
        ],
        value="ALL",
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    html.Br(),

    html.Div(dcc.Graph(id="success-pie-chart")),  # fixed ID case
    html.Br(),

    html.P("Payload range (kg):"),
    dcc.RangeSlider(
        id="payload-slider",
        min=0, max=10000, step=1000,
        marks={0: "0", 2500: "2500", 5000: "5000",
            7500: "7500", 10000: "10000"},
        value=[min_payload, max_payload]
    ),
    html.Div(dcc.Graph(id="success-payload-scatter-chart"))
])



#### TASK 2: Add a callback function to render `success-pie-chart` based on selected site dropdown

In [169]:
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(data, values='class', 
        names='pie chart names', 
        title='title')
        return fig
    else:
        filtered_df = spacex_df[spacex_df["Launch Site"] == entered_site]
        data = filtered_df.groupby("class").size().reset_index(name="count")
        fig = px.pie(
            data,
            values="count", 
            names="class",
            title=f"Success vs Failure for Site {entered_site}"
        )
    return fig


#### TASK 3: Add a callback function to render the `success-payload-scatter-chart ` scatter plot

In [170]:
@app.callback(
    Output("success-payload-scatter-chart", "figure"),
    [Input("site-dropdown", "value"),
    Input("payload-slider", "value")]
)

# Define a Function to get a Scatter Plot
def scatter(entered_site, payload):
    filtered_df = spacex_df[
        spacex_df["Payload Mass (kg)"].between(payload[0], payload[1])
    ]


    if entered_site == "ALL":
        fig = px.scatter(
            filtered_df,
            x="Payload Mass (kg)",
            y="class",
            color="Booster Version Category",
            title="Payload vs Success for All Sites"
        )
    else:
        site_df = filtered_df[filtered_df["Launch Site"] == entered_site]
        fig = px.scatter(
            site_df,
            x="Payload Mass (kg)",
            y="class",
            color="Booster Version Category",
            title=f"Payload vs Success for site {entered_site}"
        )
    return fig

In [171]:
app.run(mode='inline')


### Q1: Which site has the largest successful launches?


In [172]:
spacex_df.groupby("Launch Site")["class"].sum().sort_values(ascending=False)

Launch Site
KSC LC-39A      10
CCAFS LC-40      7
VAFB SLC-4E      4
CCAFS SLC-40     3
Name: class, dtype: int64

 KSC LC-39A has the largest number of successful launches

### Q2: Which site has the highest launch success rate?

In [173]:
success_rate = spacex_df.groupby("Launch Site")["class"].mean().sort_values(ascending=False)
success_rate


Launch Site
KSC LC-39A      0.769231
CCAFS SLC-40    0.428571
VAFB SLC-4E     0.400000
CCAFS LC-40     0.269231
Name: class, dtype: float64

KSC LC-39A   have the highest success rate (since it has both many launches and high success percentage of 76.92%)

### Q3: Which payload range(s) has the highest launch success rate?

In [174]:
spacex_df["payload_bin"] = pd.cut(spacex_df["Payload Mass (kg)"], 
                                bins=[0, 2000, 4000, 6000, 8000, 10000],
                                labels=["0–2k", "2k–4k", "4k–6k", "6k–8k", "8k–10k"])

spacex_df.groupby("payload_bin")["class"].mean().sort_values(ascending=False)


payload_bin
2k–4k     0.619048
8k–10k    0.600000
4k–6k     0.384615
0–2k      0.272727
6k–8k     0.000000
Name: class, dtype: float64

Launches with payloads between 2000–4000 kg  have the highest success rate with a percentage of 61.9%.

### Q4: Which payload range(s) has the lowest launch success rate?

Payloads over 6000 kg (6k–8k)  have the lowest success rate. 

### Q5: Which F9 Booster version has the highest launch success rate?

In [175]:
spacex_df.groupby("Booster Version")["class"].mean().sort_values(ascending=False)

Booster Version
F9 B4 B1042.1     1.0
F9 B4 B1041.1     1.0
F9 B4 B1040.1     1.0
F9 B4 B1039.1     1.0
F9 FT B1019       1.0
F9 FT  B1029.2    1.0
F9 FT  B1031.2    1.0
F9 FT  B1035.2    1.0
F9 B4 B1043.1     1.0
F9 B4 B1045.1     1.0
F9 B5  B1046.1    1.0
F9 FT  B1021.2    1.0
F9 FT B1031.1     1.0
F9 FT B1032.1     1.0
F9 FT B1026       1.0
F9 FT B1029.1     1.0
F9 FT B1025.1     1.0
F9 FT B1023.1     1.0
F9 FT B1022       1.0
F9 FT B1021.1     1.0
F9 FT B1038.1     1.0
F9 FT B1035.1     1.0
F9 FT B1036.1     1.0
F9 v1.1 B1018     1.0
F9 B4  B1039.2    0.0
F9 B4  B1040.2    0.0
F9 B4  B1041.2    0.0
F9 B4  B1043.2    0.0
F9 FT  B1038.2    0.0
F9 FT  B1036.2    0.0
F9 FT  B1032.2    0.0
F9 B4 B1044       0.0
F9 FT B1024       0.0
F9 FT B1030       0.0
F9 FT B1034       0.0
F9 FT B1020       0.0
F9 FT B1037       0.0
F9 v1.0  B0003    0.0
F9 v1.0  B0005    0.0
F9 v1.0  B0004    0.0
F9 v1.0  B0007    0.0
F9 v1.1           0.0
F9 v1.1  B1003    0.0
F9 v1.0  B0006    0.0
F9 v1.1 B1010   

FT (Falcon 9 Full Thrust) or B5 (Block 5) versions have the highest success rates, typically nearing 100% in the dataset.

-----